In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import re
import string
%matplotlib inline

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df['target'] = (df.toxic + df.severe_toxic + df.obscene + df.threat + df.insult + df.identity_hate) > 0

In [4]:
df.target.value_counts()

False    143346
True      16225
Name: target, dtype: int64

In [5]:
comments = df.comment_text
comments.head()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: comment_text, dtype: object

In [10]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
wiki_comments_tokens_raw = []
for x in comments:
    wiki_comments_tokens_raw.append(nltk.regexp_tokenize(x, pattern))

In [15]:
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list += [str(i) for i in range(10)]

wiki_comments_stopped = []
for x in wiki_comments_tokens_raw:
    wiki_comments_stopped.append([token for token in x if token not in stopwords_list])

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [21]:
count_vectorizer = CountVectorizer(stop_words=stopwords_list)
x_train_counts = count_vectorizer.fit_transform(comments)

In [23]:
tf_transformer = TfidfTransformer(use_idf=False).fit(x_train_counts)
x_train_tf = tf_transformer.transform(x_train_counts)

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [33]:
X_train, X_test, y_train, y_test= train_test_split(x_train_tf, df.target, test_size=.2)

In [34]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [38]:
y_preds = clf.predict(X_test) 

In [39]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

       False       0.92      1.00      0.96     28687
        True       0.99      0.18      0.31      3228

   micro avg       0.92      0.92      0.92     31915
   macro avg       0.95      0.59      0.63     31915
weighted avg       0.92      0.92      0.89     31915

